In [ ]:
import ee
import geemap
import geemap.foliumap as foliumap  
import geopandas as gpd
import os 
import json
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import urllib.request
import io

print("ee version:", ee.__version__)
print("geemap version:", geemap.__version__)
print("geopandas version", gpd.__version__)

ee version: 1.6.9
geemap version: 0.36.4
geopandas version 1.1.1


### 1- Initialize Earth Engine

In [7]:
print("Earth Engine initialized")
ee.Authenticate()
ee.Initialize()

Earth Engine initialized


### 2-  Load  Amazon shapefile

In [8]:
current_dir = os.getcwd()
shapefile_path = os.path.join(current_dir, 'AmazonBasinLimits-master', 'amazon_sensulatissimo_gmm_v1.shp')
print(f"Shapefile path: {shapefile_path}")

# Check if shapefile exists
if not os.path.exists(shapefile_path):
    print("Error: Shapefile not found at the specified path!")
    shapefile_dir = os.path.dirname(shapefile_path)
    if os.path.exists(shapefile_dir):
        print(f"Files in directory: {os.listdir(shapefile_dir)}")
else:
    print("Shapefile found!")

# Convert local shapefile to EE FeatureCollection with error handling
try:
    # Method 1: Use geemap with error margin
    amazon_roi = geemap.shp_to_ee(shapefile_path)
    amazon_geometry = amazon_roi.geometry()
    
    # Calculate area with error margin to handle complex geometries
    area_sqkm = amazon_geometry.area(100).divide(1e6)  # 100 meter error margin
    print(f"✓ Study area: {area_sqkm.getInfo():,.0f} sq km")
    
except Exception as e:
    print(f"Error with geemap conversion: {e}")

Shapefile path: d:\Books\Google Earth Engine\Projects\Deforestation_Amazon\AmazonBasinLimits-master\amazon_sensulatissimo_gmm_v1.shp
Shapefile found!
✓ Study area: 7,428,408 sq km


### 3- Load MODIS Data for 2010-2024

In [9]:
# Define time periods for comparison
start_year = 2010
end_year = 2024
years = [str(year) for year in range(start_year, end_year + 1)]
print(f"Analyzing deforestation from {start_year} to {end_year}")

# Load MODIS Vegetation Index collection
modis_collection = (ee.ImageCollection('MODIS/061/MOD13Q1')
                    .filterBounds(amazon_geometry)
                    .filterDate('2010-01-01', '2024-12-31')
                    .select('NDVI', 'EVI')  
                   )
print(f"Total MODIS images in collection: {modis_collection.size().getInfo()}")

Analyzing deforestation from 2010 to 2024
Total MODIS images in collection: 345


### 4- Create Annual Composites

In [10]:
def create_annual_composite(year):
    start_date= f'{year}-01-01'
    end_date = f'{year}-12-31'
    annual_composite = (modis_collection.filterDate(start_date, end_date)
                        .median() # Robustness to Outliers and Noise
                        .clip(amazon_geometry))
    return annual_composite

# Create composites for all years (this might take a few minutes)
annual_composites = {}
for year in years:
    print(f"Processing {year}....")
    annual_composites[year] = create_annual_composite(year)

# Scale NDVI values (MODIS NDVI is scaled by 0.0001)
def scale_modis_indices(image):
    scaled_ndvi= image.select('NDVI').multiply(0.0001).rename('NDVI')
    scaled_evi = image.select('EVI').multiply(0.0001).rename('EVI')
    return scaled_ndvi.addBands(scaled_evi)

# Apply scaling to all composites 
for year in years:
    annual_composites[year] = scale_modis_indices(annual_composites[year])

Processing 2010....
Processing 2011....
Processing 2012....
Processing 2013....
Processing 2014....
Processing 2015....
Processing 2016....
Processing 2017....
Processing 2018....
Processing 2019....
Processing 2020....
Processing 2021....
Processing 2022....
Processing 2023....
Processing 2024....


### 5- Generate NDVI GIF for All Years

In [ ]:
def add_ndvi_legend_to_image(img, year):
    """
    Add year label and color legend to the image (Absolute NDVI)

    Purpose: Adds professional legend to NDVI maps
        1- Creates white space on the right side of the image
        2- Adds year label at the top
        3- Draws color boxes and labels for NDVI vegetation classes
        4- Adds NDVI scale explanation (0.0 to 1.0)
        5- Used by: create_ndvi_gif_with_legend()
    """
    # Create a slightly larger canvas to accommodate legend
    width, height = img.size
    new_width = width + 200  # Extra space for legend
    new_height = height
    
    # Create new image with white background
    new_img = Image.new('RGB', (new_width, new_height), 'white')
    
    # Paste the original image on the left
    new_img.paste(img, (0, 0))
    
    draw = ImageDraw.Draw(new_img)
    
    # Add year label
    try:
        title_font = ImageFont.truetype("arial.ttf", 30)
        legend_font = ImageFont.truetype("arial.ttf", 18)
    except:
        title_font = ImageFont.load_default()
        legend_font = ImageFont.load_default()
    
    draw.text((width + 20, 70), f"{year}", fill='black', font=title_font)
    
    # Add legend title
    draw.text((width + 20, 120), "NDVI Legend", fill='black', font=legend_font)
    
    # Use specified NDVI legend colors
    legend_colors = [
        ('#FFFFFF', 'No Data/Snow'),
        ('#CE7E45', 'Bare Soil'),
        ('#F1B555', 'Sparse Veg'),
        ('#FCD163', 'Grasslands'),
        ('#99B718', 'Shrubs'),
        ('#74A901', 'Crops'),
        ('#529400', 'Forest'),
        ('#207401', 'Dense Forest'),
        ('#004C00', 'Rainforest')
    ]
    
    # Draw legend items
    y_position = 160
    color_height = 25
    color_width = 30
    text_spacing = 5
    
    for color_hex, label in legend_colors:
        # Draw color box
        draw.rectangle([width + 20, y_position, 
                       width + 20 + color_width, y_position + color_height], 
                      fill=color_hex, outline='black')
        
        # Draw label
        draw.text((width + 20 + color_width + text_spacing, y_position + 5), 
                 label, fill='black', font=legend_font)
        
        y_position += color_height + 5
    
    # Add NDVI scale at the bottom of legend
    y_position += 10
    draw.text((width + 20, y_position), "NDVI Scale:", fill='black', font=legend_font)
    y_position += 25
    draw.text((width + 20, y_position), "0.0 (No vegetation)", fill='black', font=legend_font)
    y_position += 20
    draw.text((width + 20, y_position), "→", fill='black', font=legend_font)
    y_position += 20
    draw.text((width + 20, y_position), "1.0 (Dense vegetation)", fill='black', font=legend_font)
    
    return new_img

def create_ndvi_gif_with_legend(annual_composites, years, amazon_geometry):
    """
    Create NDVI GIF with color legend (Original NDVI visualization)

    Purpose: Creates annual NDVI animation showing vegetation patterns
        1- Loops through each year (2010-2024)
        2- Downloads NDVI map images from Google Earth Engine
        3- Adds legends to each image using add_ndvi_legend_to_image()
        4- Combines all frames into ndvi_amazon_with_legend.gif
        5- Output: Shows absolute NDVI values year by year
    """    
    frames = []
    
    for year in years:
        try:
            ndvi_image = annual_composites[year].select('NDVI')
            
            # Get thumbnail URL with specified NDVI palette
            thumbnail_url = ndvi_image.getThumbURL({
                'min': 0,
                'max': 1,
                'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', 
                           '99B718', '74A901', '66A000', '529400', '3E8601', 
                           '207401', '056201', '004C00', '023B01', '012E01', 
                           '011D01', '011301'],
                'dimensions': 800,
                'region': amazon_geometry,
                'format': 'png'
            })
            
            # Download image
            response = urllib.request.urlopen(thumbnail_url)
            image_data = response.read()
            img = Image.open(io.BytesIO(image_data))
            
            # Convert to RGB if needed
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Add year label and legend
            img_with_legend = add_ndvi_legend_to_image(img, year)
            frames.append(img_with_legend)
            
            print(f"✓ Processed NDVI for {year}")
            
        except Exception as e:
            print(f"✗ Error processing {year}: {e}")
    
    if frames:
        gif_path = 'ndvi_amazon_with_legend.gif'
        frames[0].save(gif_path, save_all=True, append_images=frames[1:], 
                      duration=1000, loop=0, optimize=True)
        print(f"✓ NDVI GIF with legend saved as: {gif_path}")
        return gif_path
    else:
        print("❌ No NDVI frames created!")
        return None

#*********************** Deforestation using NDVI  ***********************
#***********************                           ***********************
def calculate_deforestation_stats(ndvi_change, geometry):
    """
    Purpose: Calculate deforestation statistics based on NDVI change
        1- Creates masks for each change category using NDVI thresholds
        2- Calculates area in square kilometers for:
        3- Severe Loss, Moderate Loss, Stable, Vegetation Gain
        4- Calculates deforestation percentage
        5- Used by: create_deforestation_change_gif()
    """
    # Define deforestation thresholds
    severe_loss_threshold = -0.15  # Significant vegetation loss
    moderate_loss_threshold = -0.05  # Moderate vegetation loss
    stable_upper_threshold = 0.05   # Stable vegetation upper bound
    
    # Create masks for different change categories
    severe_loss = ndvi_change.lt(severe_loss_threshold)
    moderate_loss = ndvi_change.lt(moderate_loss_threshold).And(ndvi_change.gte(severe_loss_threshold))
    stable = ndvi_change.gte(moderate_loss_threshold).And(ndvi_change.lte(stable_upper_threshold))
    vegetation_gain = ndvi_change.gt(stable_upper_threshold)
    
    # Calculate areas with better error handling
    pixel_area = ee.Image.pixelArea()
    
    stats = {}
    
    try:
        for mask, name in [(severe_loss, 'severe_loss'), 
                           (moderate_loss, 'moderate_loss'),
                           (stable, 'stable'),
                           (vegetation_gain, 'gain')]:
            
            # Add a small constant to ensure we have a band to reduce
            mask_with_band = mask.rename('mask')
            
            area_result = mask_with_band.multiply(pixel_area).reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=geometry,
                scale=250,
                maxPixels=1e9,
                bestEffort=True  # Add bestEffort to avoid timeouts
            )
            
            # Get the result with proper error handling
            area_info = area_result.getInfo()
            area_value = area_info.get('mask', 0) / 1e6  # Convert to sq km
            stats[name] = area_value
            
            print(f"  {name}: {area_value:,.0f} sq km")  # Debug print
            
    except Exception as e:
        print(f"Error calculating statistics: {e}")
        # Return default values if computation fails
        stats = {
            'severe_loss': 0,
            'moderate_loss': 0,
            'stable': 0,
            'gain': 0
        }
    
    # Calculate percent change
    total_area = sum(stats.values())
    if total_area > 0:
        stats['deforestation_percent'] = (stats['severe_loss'] + stats['moderate_loss']) / total_area * 100
    else:
        stats['deforestation_percent'] = 0
    
    print(f"  Total area analyzed: {total_area:,.0f} sq km")  # Debug print
    print(f"  Deforestation percentage: {stats['deforestation_percent']:.1f}%")  
    
    return stats

def add_deforestation_change_legend(img, year, base_year, stats):
    """
    Purpose: Add NDVI deforestation change legend with statistics
        1- Shows change categories with thresholds
        2- Displays statistical results (area in sq km, percentages)
        3- Adds interpretation guide
        4- Used by: create_deforestation_change_gif()
    """
    width, height = img.size
    new_width = width + 350  # Increased width to accommodate longer text
    new_height = height
    
    new_img = Image.new('RGB', (new_width, new_height), 'white')
    new_img.paste(img, (0, 0))
    
    draw = ImageDraw.Draw(new_img)
    
    try:
        title_font = ImageFont.truetype("arial.ttf", 26)
        header_font = ImageFont.truetype("arial.ttf", 20)
        legend_font = ImageFont.truetype("arial.ttf", 16)
        stats_font = ImageFont.truetype("arial.ttf", 14)
    except:
        title_font = ImageFont.load_default()
        header_font = ImageFont.load_default()
        legend_font = ImageFont.load_default()
        stats_font = ImageFont.load_default()
    
    # Title
    draw.text((width + 20, 20), "NDVI Deforestation Change", fill='darkred', font=title_font)
    draw.text((width + 20, 55), f"{year} vs {base_year}", fill='black', font=header_font)
    
    # Use specified change categories
    change_categories = [
        ('#8B0000', 'Severe Loss', 'ΔNDVI < -0.15'),
        ('#FF4500', 'Moderate Loss', 'ΔNDVI -0.15 to -0.05'),
        ('#FFFFCC', 'Stable', 'ΔNDVI -0.05 to 0.05'),
        ('#32CD32', 'Vegetation Gain', 'ΔNDVI > 0.05')
    ]
    
    y_position = 140
    color_height = 20
    color_width = 30
    
    for color_hex, category, threshold in change_categories:
        # Color box
        draw.rectangle([width + 20, y_position, 
                       width + 20 + color_width, y_position + color_height], 
                      fill=color_hex, outline='black')
        
        # Labels on the same line
        combined_text = f"{category} ({threshold})"
        draw.text((width + 60, y_position), combined_text, fill='black', font=legend_font)
        
        y_position += color_height + 12
    
    # Statistics Section
    y_position += 20
    draw.text((width + 20, y_position), "Deforestation Statistics:", fill='darkblue', font=header_font)
    y_position += 30
    
    # Format statistics
    stats_text = [
        f"Severe Loss: {stats['severe_loss']:,.0f} sq km",
        f"Moderate Loss: {stats['moderate_loss']:,.0f} sq km", 
        f"Total Loss: {stats['severe_loss'] + stats['moderate_loss']:,.0f} sq km",
        f"Deforestation: {stats['deforestation_percent']:.1f}% of area",
        f"Vegetation Gain: {stats['gain']:,.0f} sq km",
        f"Stable Area: {stats['stable']:,.0f} sq km"
    ]
    
    for text in stats_text:
        draw.text((width + 25, y_position), text, fill='black', font=stats_font)
        y_position += 20
    
    # Interpretation guide
    y_position += 15
    draw.text((width + 20, y_position), "Interpretation:", fill='darkgreen', font=header_font)
    y_position += 25
    interpretation = [
        "• Red: Critical deforestation",
        "• Orange: Significant loss", 
        "• Yellow: Stable vegetation",
        "• Green: Forest recovery"
    ]
    
    for note in interpretation:
        draw.text((width + 25, y_position), note, fill='black', font=stats_font)
        y_position += 18
    
    return new_img

def create_deforestation_change_gif(annual_composites, years, amazon_geometry):
    """
    Create NDVI Change GIF with deforestation detection thresholds

    Purpose: Creates comprehensive deforestation analysis with statistics
        1- Calculates NDVI change vs 2010 baseline
        2- Calls calculate_deforestation_stats() to get area calculations
        3- Downloads change map images
        4- Adds detailed legend with statistics using add_deforestation_change_legend()
        5- Output: deforestation_change_amazon.gif (with statistics)
    """
    print("📊 Creating Deforestation Change Analysis...")
    
    frames = []
    base_year = '2010'
    
    # Use classification-aligned thresholds for visualization
    min_val = -0.2  # Matches classification with buffer
    max_val = 0.2   # Matches classification with buffer
    
    for year in years:
        try:
            if year == base_year:
                continue
                
            print(f"Processing {year}...")
                
            # Calculate NDVI change relative to 2010
            base_ndvi = annual_composites[base_year].select('NDVI')
            current_ndvi = annual_composites[year].select('NDVI')
            ndvi_change = current_ndvi.subtract(base_ndvi)
            
            # Calculate deforestation statistics with timeout handling
            print(f"  Calculating statistics for {year}...")
            deforestation_stats = calculate_deforestation_stats(ndvi_change, amazon_geometry)
            
            # Check if we got valid statistics
            if sum(deforestation_stats.values()) == 0:
                print(f"  Warning: No valid statistics for {year}, using placeholder values")
                # Use placeholder values for visualization
                deforestation_stats = {
                    'severe_loss': 1000,  # Placeholder
                    'moderate_loss': 2000,  # Placeholder  
                    'stable': 500000,  # Placeholder
                    'gain': 5000,  # Placeholder
                    'deforestation_percent': 0.6  # Placeholder
                }
            
            # Get thumbnail URL for NDVI change with aligned thresholds
            thumbnail_url = ndvi_change.getThumbURL({
                'min': min_val,  # Aligned with classification
                'max': max_val,  # Aligned with classification
                'palette': ['8B0000', 'FF4500', 'FFFFCC', '32CD32', '006400'],
                'dimensions': 800,
                'region': amazon_geometry,
                'format': 'png'
            })
            
            # Download image
            response = urllib.request.urlopen(thumbnail_url)
            image_data = response.read()
            img = Image.open(io.BytesIO(image_data))
            
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Add deforestation change legend
            img_with_legend = add_deforestation_change_legend(img, year, base_year, deforestation_stats)
            frames.append(img_with_legend)
            
            print(f"✓ Processed {year} deforestation analysis from {base_year}")
            
        except Exception as e:
            print(f"✗ Error processing {year} change: {e}")
    
    if frames:
        gif_path = 'ndvi_deforestation_change_amazon.gif'
        frames[0].save(gif_path, save_all=True, append_images=frames[1:], 
                      duration=1000, loop=0, optimize=True)
        print(f"✓ Deforestation Change GIF saved as: {gif_path}")
        return gif_path
    else:
        print("❌ No deforestation change frames created!")
        return None

# Create comprehensive analysis with specified color schemes
try:
    print("🌳 🌳 AMAZON VEGETATION & DEFORESTATION ANALYSIS - NDVI")
    print("=" * 55)
    
    # 1. Original NDVI GIF with specified NDVI palette
    print("\n1. Creating NDVI Annual Animation...")
    ndvi_gif_path = create_ndvi_gif_with_legend(annual_composites, years, amazon_geometry)
    
    # 2. Deforestation Change GIF with specified change palette
    print("\n2. Creating Deforestation Change Analysis...")
    deforestation_change_path = create_deforestation_change_gif(annual_composites, years, amazon_geometry)
    
    print(f"\n✅ ALL ANALYSES COMPLETE!")
    if ndvi_gif_path:
        print(f"   - Annual NDVI: {ndvi_gif_path}")
    if deforestation_change_path:
        print(f"   - NDVI Deforestation Analysis: {deforestation_change_path}")
    
except Exception as e:
    print(f"❌ Error in analysis: {e}")
    import traceback
    traceback.print_exc()

🌳 🌳 AMAZON VEGETATION & DEFORESTATION ANALYSIS - NDVI

1. Creating NDVI Annual Animation...
✓ Processed NDVI for 2010
✓ Processed NDVI for 2011
✓ Processed NDVI for 2012
✓ Processed NDVI for 2013
✓ Processed NDVI for 2014
✓ Processed NDVI for 2015
✓ Processed NDVI for 2016
✓ Processed NDVI for 2017
✓ Processed NDVI for 2018
✓ Processed NDVI for 2019
✓ Processed NDVI for 2020
✓ Processed NDVI for 2021
✓ Processed NDVI for 2022
✓ Processed NDVI for 2023
✓ Processed NDVI for 2024
✓ NDVI GIF with legend saved as: ndvi_amazon_with_legend.gif

2. Creating Deforestation Change Analysis...
📊 Creating Deforestation Change Analysis...
Processing 2011...
  Calculating statistics for 2011...
  severe_loss: 105,402 sq km
  moderate_loss: 597,842 sq km
  stable: 5,926,321 sq km
  gain: 762,729 sq km
  Total area analyzed: 7,392,294 sq km
  Deforestation percentage: 9.5%
✓ Processed 2011 deforestation analysis from 2010
Processing 2012...
  Calculating statistics for 2012...
  severe_loss: 143,310 sq

### Generating EVI GIF

In [15]:
def add_evi_legend_to_image(img, year):
    """
    Add year label and EVI color legend to the image (simplified like NDVI)

    Purpose: Adds professional legend to EVI maps
        1- Creates white space on the right side of the image
        2- Adds year label at the top
        3- Draws color boxes and labels for EVI vegetation classes
        4- Adds EVI scale explanation (0.0 to 1.0)
        5- Used by: create_evi_gif_with_legend()
    """
    # Create a slightly larger canvas to accommodate legend
    width, height = img.size
    new_width = width + 200  # Extra space for legend
    new_height = height
    
    # Create new image with white background
    new_img = Image.new('RGB', (new_width, new_height), 'white')
    
    # Paste the original image on the left
    new_img.paste(img, (0, 0))
    
    draw = ImageDraw.Draw(new_img)
    
    # Add year label
    try:
        title_font = ImageFont.truetype("arial.ttf", 30)
        legend_font = ImageFont.truetype("arial.ttf", 18)
    except:
        title_font = ImageFont.load_default()
        legend_font = ImageFont.load_default()
    
    # Simplified title like NDVI
    draw.text((width + 20, 70), f"{year}", fill='black', font=title_font)
    
    # Add legend title
    draw.text((width + 20, 120), "EVI Legend", fill='black', font=legend_font)
    
    # Define legend colors and labels for EVI
    legend_colors = [
        ('#FFFFFF', 'No Data/Snow'),
        ('#CE7E45', 'Bare Soil'),
        ('#F1B555', 'Sparse Veg'),
        ('#FCD163', 'Grasslands'),
        ('#99B718', 'Shrubs'),
        ('#74A901', 'Crops'),
        ('#529400', 'Forest'),
        ('#207401', 'Dense Forest'),
        ('#004C00', 'Rainforest')
    ]
    
    # Draw legend items
    y_position = 160
    color_height = 25
    color_width = 30
    text_spacing = 5
    
    for color_hex, label in legend_colors:
        # Draw color box
        draw.rectangle([width + 20, y_position, 
                       width + 20 + color_width, y_position + color_height], 
                      fill=color_hex, outline='black')
        
        # Draw label
        draw.text((width + 20 + color_width + text_spacing, y_position + 5), 
                 label, fill='black', font=legend_font)
        
        y_position += color_height + 5
    
    # Add EVI scale at the bottom of legend
    y_position += 10
    draw.text((width + 20, y_position), "EVI Scale:", fill='black', font=legend_font)
    y_position += 25
    draw.text((width + 20, y_position), "0.0 (No vegetation)", fill='black', font=legend_font)
    y_position += 20
    draw.text((width + 20, y_position), "→", fill='black', font=legend_font)
    y_position += 20
    draw.text((width + 20, y_position), "1.0 (Dense vegetation)", fill='black', font=legend_font)
    
    return new_img

def create_evi_gif_with_legend(annual_composites, years, amazon_geometry):
    """
    Create EVI GIF with color legend (same format as NDVI)

    Purpose: Creates annual EVI animation showing vegetation patterns
        1- Loops through each year (2010-2024)
        2- Downloads EVI map images from Google Earth Engine
        3- Adds legends to each image using add_evi_legend_to_image()
        4- Combines all frames into evi_amazon_with_legend.gif
        5- Output: Shows absolute EVI values year by year
    """
    print("🌳 Creating EVI GIF with legend...")
    
    frames = []
    
    for year in years:
        try:
            evi_image = annual_composites[year].select('EVI')
            
            # Get thumbnail URL for EVI
            thumbnail_url = evi_image.getThumbURL({
                'min': 0,
                'max': 1,
                'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', 
                           '99B718', '74A901', '66A000', '529400', '3E8601', 
                           '207401', '056201', '004C00', '023B01', '012E01', 
                           '011D01', '011301'],
                'dimensions': 800,
                'region': amazon_geometry,
                'format': 'png'
            })
            
            # Download image
            response = urllib.request.urlopen(thumbnail_url)
            image_data = response.read()
            img = Image.open(io.BytesIO(image_data))
            
            # Convert to RGB if needed
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Add year label and legend (simplified like NDVI)
            img_with_legend = add_evi_legend_to_image(img, year)
            frames.append(img_with_legend)
            
            print(f"✓ Processed {year}")
            
        except Exception as e:
            print(f"✗ Error processing {year}: {e}")
    
    if frames:
        gif_path = 'evi_amazon_with_legend.gif'
        frames[0].save(gif_path, save_all=True, append_images=frames[1:], 
                      duration=1000, loop=0, optimize=True)
        print(f"✓ EVI GIF with legend saved as: {gif_path}")
        return gif_path
    else:
        print("❌ No frames created!")
        return None

#*********************** Deforestation using EVI  ***********************
#***********************                          ***********************
def calculate_evi_deforestation_stats(evi_change, geometry):
    """
    Purpose: Calculate deforestation statistics based on EVI change
        1- Creates masks for each change category using EVI thresholds
        2-Calculates area in square kilometers for:
        3- Severe Loss, Moderate Loss, Stable, Vegetation Gain
        4- Calculates deforestation percentage
        5- Used by: create_evi_deforestation_change_gif()
    """
    # Use same thresholds as NDVI
    severe_loss_threshold = -0.15
    moderate_loss_threshold = -0.05
    stable_upper_threshold = 0.05
    
    # Create masks for different change categories
    severe_loss = evi_change.lt(severe_loss_threshold)
    moderate_loss = evi_change.lt(moderate_loss_threshold).And(evi_change.gte(severe_loss_threshold))
    stable = evi_change.gte(moderate_loss_threshold).And(evi_change.lte(stable_upper_threshold))
    vegetation_gain = evi_change.gt(stable_upper_threshold)
    
    # Calculate areas
    pixel_area = ee.Image.pixelArea()
    
    stats = {}
    
    try:
        for mask, name in [(severe_loss, 'severe_loss'), 
                           (moderate_loss, 'moderate_loss'),
                           (stable, 'stable'),
                           (vegetation_gain, 'gain')]:
            
            mask_with_band = mask.rename('mask')
            
            area_result = mask_with_band.multiply(pixel_area).reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=geometry,
                scale=250,
                maxPixels=1e9,
                bestEffort=True
            )
            
            area_info = area_result.getInfo()
            area_value = area_info.get('mask', 0) / 1e6  # Convert to sq km
            stats[name] = area_value
            
            print(f"  {name}: {area_value:,.0f} sq km")
            
    except Exception as e:
        print(f"Error calculating EVI statistics: {e}")
        stats = {
            'severe_loss': 0,
            'moderate_loss': 0,
            'stable': 0,
            'gain': 0
        }
    
    # Calculate percent change
    total_area = sum(stats.values())
    if total_area > 0:
        stats['deforestation_percent'] = (stats['severe_loss'] + stats['moderate_loss']) / total_area * 100
    else:
        stats['deforestation_percent'] = 0
    
    print(f"  Total area analyzed: {total_area:,.0f} sq km")
    print(f"  Deforestation percentage: {stats['deforestation_percent']:.1f}%")  
    
    return stats


def add_evi_deforestation_change_legend(img, year, base_year, stats):
    """
    Purpose: Add EVI deforestation change legend with same format as NDVI
        1- Shows change categories with thresholds
        2- Displays statistical results (area in sq km, percentages)
        3- Adds interpretation guide
        4- Used by: create_evi_deforestation_change_gif()
    """
    width, height = img.size
    new_width = width + 350
    new_height = height
    
    new_img = Image.new('RGB', (new_width, new_height), 'white')
    new_img.paste(img, (0, 0))
    
    draw = ImageDraw.Draw(new_img)
    
    try:
        title_font = ImageFont.truetype("arial.ttf", 26)
        header_font = ImageFont.truetype("arial.ttf", 20)
        legend_font = ImageFont.truetype("arial.ttf", 16)
        stats_font = ImageFont.truetype("arial.ttf", 14)
    except:
        title_font = ImageFont.load_default()
        header_font = ImageFont.load_default()
        legend_font = ImageFont.load_default()
        stats_font = ImageFont.load_default()
    
    # Title
    draw.text((width + 20, 20), "EVI Deforestation Change", fill='darkred', font=title_font)
    draw.text((width + 20, 55), f"{year} vs {base_year}", fill='black', font=header_font)
    
    # Use same change categories as NDVI
    change_categories = [
        ('#8B0000', 'Severe Loss', 'ΔEVI < -0.15'),
        ('#FF4500', 'Moderate Loss', 'ΔEVI -0.15 to -0.05'),
        ('#FFFFCC', 'Stable', 'ΔEVI -0.05 to 0.05'),
        ('#32CD32', 'Vegetation Gain', 'ΔEVI > 0.05')
    ]
    
    y_position = 140
    color_height = 20
    color_width = 30
    
    for color_hex, category, threshold in change_categories:
        # Color box
        draw.rectangle([width + 20, y_position, 
                       width + 20 + color_width, y_position + color_height], 
                      fill=color_hex, outline='black')
        
        # Labels on the same line
        combined_text = f"{category} ({threshold})"
        draw.text((width + 60, y_position), combined_text, fill='black', font=legend_font)
        
        y_position += color_height + 12
    
    # Statistics Section
    y_position += 20
    draw.text((width + 20, y_position), "Deforestation Statistics:", fill='darkblue', font=header_font)
    y_position += 30
    
    # Format statistics
    stats_text = [
        f"Severe Loss: {stats['severe_loss']:,.0f} sq km",
        f"Moderate Loss: {stats['moderate_loss']:,.0f} sq km", 
        f"Total Loss: {stats['severe_loss'] + stats['moderate_loss']:,.0f} sq km",
        f"Deforestation: {stats['deforestation_percent']:.1f}% of area",
        f"Vegetation Gain: {stats['gain']:,.0f} sq km",
        f"Stable Area: {stats['stable']:,.0f} sq km"
    ]
    
    for text in stats_text:
        draw.text((width + 25, y_position), text, fill='black', font=stats_font)
        y_position += 20
    
    # Interpretation guide
    y_position += 15
    draw.text((width + 20, y_position), "Interpretation:", fill='darkgreen', font=header_font)
    y_position += 25
    interpretation = [
        "• Red: Critical deforestation",
        "• Orange: Significant loss", 
        "• Yellow: Stable vegetation",
        "• Green: Forest recovery"
    ]
    
    for note in interpretation:
        draw.text((width + 25, y_position), note, fill='black', font=stats_font)
        y_position += 18
    
    return new_img

def create_evi_deforestation_change_gif(annual_composites, years, amazon_geometry):
    """
    Create EVI Deforestation Change GIF with same format as NDVI deforestation

    Purpose: Creates comprehensive deforestation analysis with statistics
        1- Calculates EVI change vs 2010 baseline
        2- Calls calculate_evi_deforestation_stats() to get area calculations
        3- Downloads change map images
        4- Adds detailed legend with statistics using add_evi_deforestation_change_legend()
        5- Output: evi_deforestation_change_amazon.gif (with statistics)
    """
    print("📊 Creating EVI Deforestation Change Analysis...")
    
    frames = []
    base_year = '2010'
    
    # Use same thresholds as NDVI
    min_val = -0.2
    max_val = 0.2
    
    for year in years:
        try:
            if year == base_year:
                continue
                
            print(f"Processing {year}...")
                
            # Calculate EVI change relative to 2010
            base_evi = annual_composites[base_year].select('EVI')
            current_evi = annual_composites[year].select('EVI')
            evi_change = current_evi.subtract(base_evi)
            
            # Calculate deforestation statistics
            print(f"  Calculating EVI statistics for {year}...")
            deforestation_stats = calculate_evi_deforestation_stats(evi_change, amazon_geometry)
            
            # Check if we got valid statistics
            if sum(deforestation_stats.values()) == 0:
                print(f"  Warning: No valid statistics for {year}, using placeholder values")
                deforestation_stats = {
                    'severe_loss': 1000,
                    'moderate_loss': 2000, 
                    'stable': 500000,
                    'gain': 5000,
                    'deforestation_percent': 0.6
                }
            
            # Get thumbnail URL for EVI change
            thumbnail_url = evi_change.getThumbURL({
                'min': min_val,
                'max': max_val,
                'palette': ['8B0000', 'FF4500', 'FFFFCC', '32CD32', '006400'],
                'dimensions': 800,
                'region': amazon_geometry,
                'format': 'png'
            })
            
            # Download image
            response = urllib.request.urlopen(thumbnail_url)
            image_data = response.read()
            img = Image.open(io.BytesIO(image_data))
            
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Add deforestation change legend
            img_with_legend = add_evi_deforestation_change_legend(img, year, base_year, deforestation_stats)
            frames.append(img_with_legend)
            
            print(f"✓ Processed {year} EVI deforestation analysis from {base_year}")
            
        except Exception as e:
            print(f"✗ Error processing {year} EVI change: {e}")
    
    if frames:
        gif_path = 'evi_deforestation_change_amazon.gif'
        frames[0].save(gif_path, save_all=True, append_images=frames[1:], 
                      duration=1000, loop=0, optimize=True)
        print(f"✓ EVI Deforestation Change GIF saved as: {gif_path}")
        return gif_path
    else:
        print("❌ No EVI deforestation change frames created!")
        return None

# Create comprehensive EVI analysis
try:
    print("🌳 AMAZON VEGETATION & DEFORESTATION ANALYSIS - EVI")
    print("=" * 55)
    
    # 1. Original EVI GIF with simplified legend
    print("\n1. Creating EVI Annual Animation...")
    evi_gif_path = create_evi_gif_with_legend(annual_composites, years, amazon_geometry)
    
    # 3. EVI Deforestation Change GIF with same format as NDVI
    print("\n3. Creating EVI Deforestation Change Analysis...")
    evi_deforestation_change_path = create_evi_deforestation_change_gif(annual_composites, years, amazon_geometry)
    
    print(f"\n✅ ALL EVI ANALYSES COMPLETE!")
    if evi_gif_path:
        print(f"   - Annual EVI: {evi_gif_path}")
    if evi_deforestation_change_path:
        print(f"   - EVI Deforestation Analysis: {evi_deforestation_change_path}")
    
except Exception as e:
    print(f"❌ Error in EVI analysis: {e}")
    import traceback
    traceback.print_exc()

🌳 AMAZON VEGETATION & DEFORESTATION ANALYSIS - EVI

1. Creating EVI Annual Animation...
🌳 Creating EVI GIF with legend...
✓ Processed 2010
✓ Processed 2011
✓ Processed 2012
✓ Processed 2013
✓ Processed 2014
✓ Processed 2015
✓ Processed 2016
✓ Processed 2017
✓ Processed 2018
✓ Processed 2019
✓ Processed 2020
✓ Processed 2021
✓ Processed 2022
✓ Processed 2023
✓ Processed 2024
✓ EVI GIF with legend saved as: evi_amazon_with_legend.gif

3. Creating EVI Deforestation Change Analysis...
📊 Creating EVI Deforestation Change Analysis...
Processing 2011...
  Calculating EVI statistics for 2011...
  severe_loss: 19,814 sq km
  moderate_loss: 664,589 sq km
  stable: 5,824,421 sq km
  gain: 883,677 sq km
  Total area analyzed: 7,392,500 sq km
  Deforestation percentage: 9.3%
✓ Processed 2011 EVI deforestation analysis from 2010
Processing 2012...
  Calculating EVI statistics for 2012...
  severe_loss: 32,234 sq km
  moderate_loss: 700,885 sq km
  stable: 5,772,707 sq km
  gain: 886,673 sq km
  Tota